In [26]:
from collections import defaultdict

import codecs
import gensim
import networkx as nx
import numpy as np

from bs4 import BeautifulSoup

In [11]:
G = nx.read_gexf("datasets/influences.gexf")
names = [ n for n in G.nodes() ]

In [91]:
import logging
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

def philosopher2vec(name):
  def get_text(name):
    txt = codecs.open("datasets/pages/%s.html" % name,"r", "utf-8-sig").read()

    txt = BeautifulSoup(txt, "html.parser")
    txt = txt.get_text()

    return txt
  
  def tokenize(sentences):
    stoplist = set(codecs.open("datasets/stopwords.txt","r", "utf-8-sig").read().split())
    texts = [[word.replace(",", "") for word in document.lower().split() if word not in stoplist] for document in sentences]

    frequency = defaultdict(int)

    for text in texts:
      for token in text:
        frequency[token] += 1
    
    texts = [[token for token in text if frequency[token] > 1] for text in texts]
    
    return texts
  
  def to_dict(tokens):
    d = gensim.corpora.Dictionary(tokens)

    return d
  
  def to_corpus(d, t):
    c = [d.doc2bow(text) for text in t]
    
    return c

  txt = get_text(name)
  sentences = txt.split(".")
  
  tokens = tokenize(sentences)
  dictionary = to_dict(tokens)
  
  corpus = to_corpus(dictionary, tokens)

  bigrams = gensim.models.Phrases(tokens)

  lsi = gensim.models.lsimodel.LsiModel(corpus, id2word=dictionary, num_topics=300)
  word2vec = gensim.models.Word2Vec(bigrams[tokens], size=100, window=5, min_count=5, workers=4)

  # print tokens
  
  print name
  print "sentences: %s"% len(sentences)
  print "tokens: " % np.sum([ len(t) for t in tokens ])
  print dictionary
  # print bigrams
  # print lsi.print_topics(10)
  print
  
  for (u,s) in word2vec.most_similar(positive=['work', "philosophy"]):
    print "%s: %s" % (u,s)
    
  print

#names = names[0:10]
names = ["Karl_Popper", "Martin_Heidegger", "Ludwig_Wittgenstein"]
# names = ["Ludwig_Wittgenstein"]
  
[ philosopher2vec(name) for name in names ];

Karl_Popper
sentences: 923
tokens: 
Dictionary(1439 unique tokens: [u'', u'searle', u'writings', u'four', u'whose']...)

theory: 0.746167421341
science: 0.70251262188
popper: 0.699823260307
scientific: 0.691341757774
history: 0.661367237568
social: 0.660380244255
popper's: 0.650854229927
open_society: 0.645365357399
may: 0.639148950577
life: 0.629142403603

Martin_Heidegger


sentences: 1007
tokens: 
Dictionary(1566 unique tokens: [u'', u'limited', u'writings', u"lacoue-labarthe's", u'bedeutungslehre']...)

heidegger's: 0.960094630718
heidegger: 0.958546280861
being: 0.944669008255
phenomenology: 0.927272081375
theory: 0.926687538624
nader_el-bizri: 0.925215005875
history: 0.925126850605
thought: 0.923619270325
^: 0.917414903641
published: 0.915873765945

Ludwig_Wittgenstein
sentences: 1710
tokens: 
Dictionary(1922 unique tokens: [u'', u'searle', u'writings', u'augustine', u'linz[edit]']...)

wittgenstein: 0.896718621254
theory: 0.864893436432
language: 0.859470129013
cambridge: 0.846630096436
wittgenstein's: 0.829942703247
philosophical: 0.822431564331
ludwig: 0.813782393932
moore: 0.810309410095
philosophical_investigations: 0.802277743816
logical: 0.801936507225

